In [4]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


data = pd.read_csv('Expresso_churn_dataset.csv')
df = data.copy()
# # df.drop('user_id', axis = 1, inplace = True)
# # for i in df:
# #     if df[i].isnull().sum() / len(df) * 100 > 50:
# #         df.drop(i, axis = 1, inplace = True)

# # sampling0 = df[df.CHURN == 0]
# # sampling0 = sampling0.dropna()
# # sampling0.reset_index(drop = True, inplace = True)
# # sampling1 = df[df.CHURN == 1]
# # sampling1.drop(['REGION', 'TOP_PACK', 'FREQ_TOP_PACK'], axis = 1, inplace = True)
# # sampling1.dropna(inplace = True)
# # sampling1.reset_index(drop = True, inplace =True)

# # sampling0 = sampling0.sample(35000)
# # cols = sampling1.columns
# # dx = pd.concat([sampling1, sampling0[cols]], axis = 0)
# # df = dx.copy()
# # df.dropna()

categoricals = df.select_dtypes(include = ['object', 'category'])
numericals = df.select_dtypes(include = 'number')

def outlierRemoval(dataframe):
    for i in dataframe.columns:
        if dataframe[i].dtypes != 'O': # --------------------------------------- If the data type is not an object type
            Q1 = dataframe[i].describe()[4] # ---------------------------------- Identify lower Quartile
            Q3 = dataframe[i].describe()[6] # ---------------------------------- Identify the upper quartile
            IQR = Q3 - Q1 # ---------------------------------------------------- Get Inter Quartile Range
            minThreshold = Q1 - 1.5 * IQR # ------------------------------------ Get Minimum Threshold
            maxThreshold = Q3 + 1.5 * IQR # ------------------------------------ Get Maximum Threshold

            dataframe = dataframe.loc[(dataframe[i] >= minThreshold) & (dataframe[i] <= maxThreshold)]
    return dataframe


df = outlierRemoval(df)

from sklearn.preprocessing import StandardScaler, LabelEncoder
scaler = StandardScaler()
encoder = LabelEncoder()

for i in numericals.columns: # ................................................. Select all numerical columns
    if i in df.drop('CHURN', axis = 1).columns: # ...................................................... If the selected column is found in the general dataframe
        df[i] = scaler.fit_transform(df[[i]]) # ................................ Scale it

for i in categoricals.columns: # ............................................... Select all categorical columns
    if i in df.drop('CHURN', axis = 1).columns: # ...................................................... If the selected columns are found in the general dataframe
        df[i] = encoder.fit_transform(df[i])# .................................. encode it

df.head()

sel_cols = ['REGULARITY', 'DATA_VOLUME','REVENUE',  'ORANGE', 'ON_NET', 'MONTANT','FREQUENCE']
x = df[sel_cols]

x = x
y = df.CHURN
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.20, random_state = 75, stratify = y)


# Modelling
model = RandomForestClassifier() 
model.fit(xtrain, ytrain) 
cross_validation = model.predict(xtrain)
pred = model.predict(xtest) 

# save model
model = pickle.dump(model, open('Expresso_churn.pkl', 'wb'))
print('\nModel is saved\n')



Model is saved

